# WEB SCRAPING AND SUMMERIZATION

Web Scraping and summarization of its content by using a DeepSeek API.

In [ ]:
# import libraries

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup  #web scraping library
from IPython.display import Markdown, display
from openai import OpenAI

# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [2]:
# initialization of OpenAI client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)


# making a chat completation request
completion = client.chat.completions.create(
  model="deepseek/deepseek-r1-zero:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

\boxed{The question regarding the meaning of life is a deeply philosophical and subjective one that varies based on one's cultural, religious, philosophical, and personal beliefs. Common responses include seeking personal fulfillment through relationships and passions, religious faith which often entails serving a higher power or following a path laid down by religious teachings, existentialist ideas which suggest that individuals must create their own meaning, and biological viewpoints which focus on survival and reproduction. However, there is no single, universally accepted answer that addresses what the meaning of life is for every individual.}


In [4]:
# PROJECT 1 : TEXT SUMMERIZATION FROM A WEBPAGE 

# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites block requests from unknown risources. The header mimics a real browser request to 
# avoid being blocked when making HTTP requests

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# the class reppresents a webpage
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser') # it extracts data from HTML and XML
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [6]:
# define the system prompt for the model 

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:
# += because otherwise it will overwrite the previous text

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

MESSAGES

The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

[

    {"role": "system", "content": "system message goes here"},
    
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [11]:
def messages_for(website):
    return [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_for(website)}
    ]

In [12]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [15]:
# BRING EVERYTHING TOGETHER

def summarize(url):
    website = Website(url)
    response = client.chat.completions.create(
        model = "deepseek/deepseek-r1-zero:free",
        messages = messages_for(website)
    )
    return response.choices[0].message.content


In [16]:
summarize("https://edwarddonner.com")

'\\boxed{\n### Summary of Edward Donner\'s Website\n\n#### Introduction\nEdward Donner is a co-founder and CTO of Nebula.io, a company that uses AI to help people discover their potential and pursue their career goals. He previously founded and was the CEO of untapt, which was acquired in 2021. He enjoys writing code, experimenting with LLMs, DJing, and amateur electronic music production.\n\n#### Projects\n- **Connect Four**: An arena where LLMs (Large Language Models) are pitted against each other in a "battle of diplomacy and deviousness."\n\n#### Recent Posts\n1. **LLM Workshop – Hands-on with Agents – resources** (January 23, 2025)\n2. **Welcome, SuperDataScientists!** (December 21, 2024)\n3. **Mastering AI and LLM Engineering – Resources** (November 13, 2024)\n4. **From Software Engineer to AI Data Scientist – resources** (October 16, 2024)\n\n#### Contact Information\n- Email: ed [at] edwarddonner [dot] com\n- Website: www.edwarddonner.com\n- Social Media: LinkedIn, Twitter, Fac

In [17]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

\boxed{
```markdown
### Summary of "Home - Edward Donner"

**Introduction:**
Edward Donner is a co-founder and CTO of Nebula.io, a company that uses AI to help people discover their potential and pursue their reason for being. He previously founded and was the CEO of untapt, which was acquired in 2021. He enjoys writing code, experimenting with LLMs, DJing, and amateur electronic music production.

**Projects:**
- **Connect Four**: A project or game (details not specified on the main page).
- **Outsmart**: An arena where LLMs compete against each other in a "battle of diplomacy and deviousness."

**Blog Posts:**
1. "LLM Workshop – Hands-on with Agents – resources" (January 23, 2025)
2. "Welcome, SuperDataScientists!" (December 21, 2024)
3. "Mastering AI and LLM Engineering – Resources" (November 13, 2024)
4. "From Software Engineer to AI Data Scientist – resources" (October 16, 2024)

**Contact:**
- Email: ed [at] edwarddonner [dot] com
- Website: www.edwarddonner.com

**Social Media:**
- LinkedIn, Twitter, and Facebook links are provided.

**Newsletter Subscription:**
Visitors can subscribe to a newsletter by entering their email address.

This website serves as a personal and professional profile for Edward Donner, highlighting his work, interests, and providing a way to connect with him.
```
}

In [19]:
# LET'S TRY WITH OTHER WEBSITES
display_summary("https://cnn.com")

\boxed{
This CNN website is a comprehensive news source covering a wide variety of topics such as US and world news, politics, business, health, entertainment, style, travel, sports, science, climate, and weather. 

#### Top headlines include:
1. **Myanmar Earthquake**: Over 150 people have died and more than 730 were injured after a 7.7-magnitude earthquake hit Myanmar. Authorities have made a rare plea for help as the death toll is expected to rise. Additionally, there were visual reports capturing the collapse of a Bangkok skyscraper and pool water spilling over the edges of a highrise building during the earthquake.
2. **Trump and Greenland**: Trump stated that the US should aim to "have Greenland" as US Senator JD Vance visited the island. 
3. **State Department & USAID**: The State Department has formally notified Congress that it is effectively dissolving the United States Agency for International Development (USAID).
4. **Canada's Warning on Tariffs**: Canada has warned that it will retaliate against Trump's proposed tariffs.
5. **Scientific Breakthrough**: Scientists have found a new possibility for how life on Earth could have started through a redone experiment.
6. **Olympic Skier's Death**: Olympic skier Berkin Usta, 24, and his father died in a hotel fire.
7. **Apple's Plans**: Apple intends to "reinvigorate interest" with an overhaul of the iPhone design.

#### Features and Sections:
- **Israel's Military Actions**: Israel strikes southern Beirut for the first time since a November ceasefire.
- **Extremist Views in Israel**: Rising extremist views among young Jewish Israelis mirror societal divisions.
- **Social Media Trends**: Studio Ghibli-style AI portraits are trending on social media.
- **Panama Canal**: The sale of Panama Canal ports has been put on hold by Chinese regulators.
- **Floods in Australia**: "Unprecedented" floods have left vast areas of Australia under water.
- **Health and Wellness**: A short video features a young mom speaking out about her fentanyl addiction.
- **Travel and Lifestyle**: There are articles about staying in a cemetery in France and an interview with a conservationist who has been in Uganda’s Bwindi Impenetrable National Park for 30 years.
- **Gaming and Technology**: Studies suggest that one thing might reduce phone usage by half.

#### Additional Stories:
- **Fashion**: H&M plans to use AI clones of its human models.
- **Crime Reporting**: A child’s complaint led a babysitter to discover a man hiding under the bed.
- **Tesla Concerns**: Investors question if Musk is dedicating enough time to Tesla amidst various controversies.

#### Featured Sections:
- **Space and Science**: Articles include a newly-discovered creature preserved in 99 million-year-old amber and a study on toxic dust on Mars.
- **Global Travel**: Highlights include a massive collection of vintage cars stored in a desert and an article on why Japan’s hot springs are running low due to overtourism.

#### Extreme Weather and Disaster Information:
Specific updates on weather events such as the floods in Australia and Myanmar’s earthquake are given prominent coverage along with advice and safety tips relevant to those events.

#### Other Miscellaneous News:
- **US Politics**: Includes details on Trump's executive orders and his plans for the deportation flight details.
- **Health and Wellness**: Advice on maintaining sustainable health habits, the trend of hot workouts such as heated workouts and hot yoga, and why people wake up to pee at night.
- **Sports News**: An exciting halfcourt buzzer-beater by Josh Giddey against LeBron James and Tyler O’Neill’s record-extending sixth Opening Day home run.
- **Celebrity News Highlights**: Jeff Bezos and his fiancée planning a wedding in Venice, Megan Fox and Machine Gun Kelly welcoming a baby girl, and Will Smith making references to his Oscar slap in his new music.

This CNN website offers a broad and in-depth look at current events, making it a one-stop-shop for a wide range of news topics.
}

In [20]:
# Do it with Selenium

display_summary("https://anthropic.com")

\boxed{
```markdown
# Just a moment...

This website requires JavaScript and cookies to be enabled in order to access its contents. The "Just a moment..." message typically indicates that a security check is being performed, which is common for websites that use a service like Cloudflare to protect against automated bots. Ensure that both JavaScript and cookies are enabled in your browser to proceed.
```
}